In [2]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import pytorch_lightning as pl

#d2l
import d2l.torch as d2l

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter


#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For Progress Bar
from tqdm import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Improvement
+ relation not just one
+ add base class

## Dataset set up
+ NELL one: 30 base classes, 20 novel classes(5 way 5 shot 4 sessions)

## Dataset and DataLoader
+ `few`: instance number in support set"

In [37]:
import json
import random

import numpy as np

dataset = dict()
tail = ''
data_dir = {
    'train_tasks_in_train': '/train_tasks_in_train.json',
    'train_tasks': '/train_tasks.json',
    'test_tasks': '/test_tasks.json',
    'dev_tasks': '/dev_tasks.json',

    'rel2candidates_in_train': '/rel2candidates_in_train.json',
    'rel2candidates': '/rel2candidates.json',

    'e1rel_e2_in_train': '/e1rel_e2_in_train.json',
    'e1rel_e2': '/e1rel_e2.json',

    'ent2ids': '/ent2ids',
    'ent2vec': '/ent2vec.npy',
}

for k, v in data_dir.items():
    data_dir[k] = './NELL' + v

In [45]:
from data_loader import DataLoader

params = {'few': 1, 'batch_size': 2, 'num_query': 3}
dataset['train_tasks'] = json.load(open(data_dir['train_tasks' + tail]))  # (h_i, r, e_i) only one relation in a current task
dataset['test_tasks'] = json.load(open(data_dir['test_tasks']))
dataset['dev_tasks'] = json.load(open(data_dir['dev_tasks']))  #
dataset['rel2candidates'] = json.load(open(data_dir['rel2candidates' + tail]))  # negative t candidate 
dataset['e1rel_e2'] = json.load(open(data_dir['e1rel_e2' + tail]))  # the whole graph key: e1rel value e2
dataset['ent2id'] = json.load(open(data_dir['ent2ids']))    # entity id
dataset['ent2emb'] = np.load(data_dir['ent2vec'])   # entity embedding

train_data_loader = DataLoader(dataset, params, step='train')

In [99]:
dataset['train_tasks']

{'concept:leaguecoaches': [['concept:sportsleague:mlb',
   'concept:leaguecoaches',
   'concept:coach:joe_torre'],
  ['concept:sportsleague:mlb',
   'concept:leaguecoaches',
   'concept:personmexico:curtis_granderson'],
  ['concept:sportsleague:mlb',
   'concept:leaguecoaches',
   'concept:coach:seth_smith'],
  ['concept:sportsleague:mlb',
   'concept:leaguecoaches',
   'concept:coach:john_mcgraw'],
  ['concept:sportsleague:mlb',
   'concept:leaguecoaches',
   'concept:personmexico:jack_morris'],
  ['concept:sportsleague:nhl',
   'concept:leaguecoaches',
   'concept:coach:john_fox'],
  ['concept:sportsleague:nhl',
   'concept:leaguecoaches',
   'concept:coach:bruce_boudreau'],
  ['concept:sportsleague:nhl',
   'concept:leaguecoaches',
   'concept:coach:kenny_natt'],
  ['concept:sportsleague:nhl',
   'concept:leaguecoaches',
   'concept:coach:terry_murray'],
  ['concept:sportsleague:nhl',
   'concept:leaguecoaches',
   'concept:personmexico:paul_byrd'],
  ['concept:sportsleague:nhl',
  

### construct support and query
random choose support(相同数量 positive and negative) query(相同数量 positive and negative) according current relation

In [50]:
few = 1
nq = 3
curr_rel_idx = 0
all_rels = sorted(list(dataset['train_tasks'].keys()))
rel2candidates = dataset['rel2candidates']
tasks = dataset['train' + '_tasks']
curr_rel = all_rels[curr_rel_idx]
curr_rel_idx = curr_rel_idx + 1
curr_cand = rel2candidates[curr_rel]  # relation candidate?
e1rel_e2 = dataset['e1rel_e2']

In [38]:
# random choose support and query according current relation
curr_tasks = tasks[curr_rel]
curr_tasks_idx = np.arange(0, len(curr_tasks), 1)
curr_tasks_idx = np.random.choice(curr_tasks_idx, few + nq)
support_triples = [curr_tasks[i] for i in curr_tasks_idx[:few]]
query_triples = [curr_tasks[i] for i in curr_tasks_idx[few:]]

In [84]:
# construct support and query negative triples(num same as positive)
support_negative_triples = []
for triple in support_triples:
    e1, rel, e2 = triple
    while True:
        negative = random.choice(curr_cand)
        if (negative not in e1rel_e2[e1 + rel]) \
                and negative != e2:
            break
    support_negative_triples.append([e1, rel, negative])

negative_triples = []
for triple in query_triples:
    e1, rel, e2 = triple
    while True:
        negative = random.choice(curr_cand)
        if (negative not in e1rel_e2[e1 + rel]) \
                and negative != e2:
            break
    negative_triples.append([e1, rel, negative])
support_triples, support_negative_triples, query_triples, negative_triples

([['concept:academicfield:technology',
   'concept:academicfieldsuchasacademicfield',
   'concept:academicfield:design']],
 [['concept:academicfield:technology',
   'concept:academicfieldsuchasacademicfield',
   'concept:academicfield:general_education']],
 [['concept:academicfield:science',
   'concept:academicfieldsuchasacademicfield',
   'concept:academicfield:philosophy'],
  ['concept:academicfield:international_business',
   'concept:academicfieldsuchasacademicfield',
   'concept:academicfield:business'],
  ['concept:academicfield:technology',
   'concept:academicfieldsuchasacademicfield',
   'concept:academicfield:civil_engineering_degree']],
 [['concept:academicfield:science',
   'concept:academicfieldsuchasacademicfield',
   'concept:academicfield:literatures'],
  ['concept:academicfield:international_business',
   'concept:academicfieldsuchasacademicfield',
   'concept:academicfield:church_history'],
  ['concept:academicfield:technology',
   'concept:academicfieldsuchasacademi

In [89]:
train_task, curr_rel = train_data_loader.next_batch()   # train task consist of batch_size support, support_negative, query, query_negative

In [90]:
support_triples, support_negative_triples, query_triples, negative_triples = train_task # batch_size * num

In [95]:
support_triples[1].append(support_triples[0][0])    # stimulate base class
# support_negative_triples[0].append(support_negative_triples[0][0])
# query_triples[0].append(query_triples[0][0])
# negative_triples[0].append(negative_triples[0][0])
# train_task = support_triples, support_negative_triples, query_triples, negative_triples

In [96]:
support_triples

([['concept:stateorprovince:new_york_state',
   'concept:stateorprovinceoforganizationheadquarters',
   'concept:city:carolina'],
  ['concept:stateorprovince:new_york_state',
   'concept:stateorprovinceoforganizationheadquarters',
   'concept:city:carolina']],
 [['concept:airport:harrisburg_international',
   'concept:airportincity',
   'concept:city:harrisburg'],
  ['concept:stateorprovince:new_york_state',
   'concept:stateorprovinceoforganizationheadquarters',
   'concept:city:carolina']])

In [70]:
from models import MetaR

# test shape
param = {'dataset': 'NELL-One', 'data_path': './NELL', 'data_form': 'Pre-Train', 'seed': None, 'few': 1, 'num_query': 3, 'metric': 'MRR', 'embed_dim': 100, 'batch_size': 1024, 'learning_rate': 0.001, 'early_stopping_patience': 30, 'epoch': 100000, 'print_epoch': 100, 'eval_epoch': 1000, 'checkpoint_epoch': 1000,
                                                                                                                                                    'beta': 5, 'margin': 1, 'dropout_p': 0.5, 'ablation': False, 'device': torch.device(type='cpu'), 'prefix': 'nellone_1shot_pretrain', 'step': 'train', 'log_dir'
 : 'log', 'state_dir': 'state', 'eval_ckpt': None, 'eval_by_rel': False}
metaR = MetaR(dataset, param)
p_score, n_score = metaR(train_task, False, '')

ValueError: expected sequence of length 2 at dim 1 (got 1)

In [98]:
from embedding import Embedding
embedding = Embedding(dataset, param)
embedding(support_triples).shape

torch.Size([2, 2, 2, 100])

## MetaR train

### embedding entity
get embeddings according to entity id 

In [98]:
from embedding import Embedding


parameter = {'device': 'cpu', 'embed_dim': 100, 'data_form': 'Pre-Train'}
embedding = Embedding(dataset, parameter)
support, support_negative, query, negative = [embedding(t) for t in train_task]

support.shape, query.shape, support_negative.shape  # shape (batch_size, num, e1_e2, embedding_shape)

(torch.Size([8, 1, 2, 100]),
 torch.Size([8, 3, 2, 100]),
 torch.Size([8, 1, 2, 100]))

### relation learner
![](imgs/img.png)
![](imgs/img_4.png)

In [110]:
from models import RelationMetaLearner

# learn relation(FC)
relation_learner = RelationMetaLearner(1, embed_size=100, num_hidden1=500, num_hidden2=200, out_size=100, dropout_p=0.5)
rel = relation_learner(support)    # FC
few = 1
num_sn = 1
rel_s = rel.expand(-1, few+num_sn, -1, -1)
rel.shape, rel_s.shape

(torch.Size([8, 1, 1, 100]), torch.Size([8, 2, 1, 100]))

### gradient descent in support 
![](imgs/img_1.png)
![](imgs/img_2.png)
![](imgs/img_3.png)

In [121]:
def split_concat(positive, negative):
    pos_neg_e1 = torch.cat([positive[:, :, 0, :],
                            negative[:, :, 0, :]], 1).unsqueeze(2)
    pos_neg_e2 = torch.cat([positive[:, :, 1, :],
                            negative[:, :, 1, :]], 1).unsqueeze(2)
    return pos_neg_e1, pos_neg_e2
sup_neg_e1, sup_neg_e2 = split_concat(support, support_negative)
sup_neg_e1.shape
score = -torch.norm(sup_neg_e1 + rel_s - sup_neg_e2, 2, -1).squeeze(2)
score.shape

torch.Size([8, 2, 1, 100])

In [20]:
loss_func = nn.MarginRankingLoss(0)
loss_func(torch.ones(1024, 1), torch.ones(1024, 1), torch.ones(1))


tensor(0.)

In [21]:
torch.ones(1024, 1)

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])